# Задание 1

Используя OpenAI API попробуйте сделать следующие вещи:

1) Попробуйте воспроизвести частеречную разметку с помощью gpt-4o-mini. Для данного предложения попробуйте получить точно такую же разметку.
   Если не работает, то попробуйте добавить примеров из opencorpora (не добавляя этот пример). Если у вас сработало это предложение, то попробуйте любое другое и оцените насколько промпт обобщает.  
   !Не размечайте большое количество предложений в датасете и не рассчитывайте метрики качества, оценивайте только вручную

In [ ]:
sentence = "Великолепная «Школа злословия» вернулась в эфир после летних каникул в новом формате."
sentence_pos = [['Великолепная', 'ADJF'],
                ['«', 'PNCT'],
                ['Школа', 'NOUN'],
                ['злословия', 'NOUN'],
                ['»', 'PNCT'],
                ['вернулась', 'VERB'],
                ['в', 'PREP'],
                ['эфир', 'NOUN'],
                ['после', 'PREP'],
                ['летних', 'ADJF'],
                ['каникул', 'NOUN'],
                ['в', 'PREP'],
                ['новом', 'ADJF'],
                ['формате', 'NOUN'],
                ['.', 'PNCT']]

Мы будем загружать API-ключ из переменной среды. Для запуска кода нужно, чтобы рядом лежал `.env`-файл, содержащий ключ в переменной `OPENAI_API_KEY`, либо эта переменная иным способом была добавлена в окружение.

In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv


load_dotenv()

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

Системный промпт будет незатейливым. Мы присваиваем модели роль лингвиста-разметчика, описываем, как требуется сделать токенизацию и частеречную разметку, формат выходных данных, а также задаём список тегов OpenCorpora, которые нужно использовать. Теги и примеры для промпта я взял [Здесь](https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html)

In [ ]:
with open('hw5_system_prompt.txt', 'r', encoding='utf-8') as f:
    system_prompt = f.read()

print(system_prompt)

You are an expert linguist specializing in Part-of-Speech (POS) tagging for the Russian language.

Your task is to analyze a given Russian sentence. First, tokenize the sentence, ensuring that words and punctuation marks (such as «, », ., etc.) are treated as separate tokens. Then, for each token, assign the appropriate Part-of-Speech tag from the OpenCorpora tagset provided below. The output should be a JSON with a single key `tokens`: list of lists, where each inner list contains the token and its corresponding tag. The response should contain only this structure, nothing additional.

You must only use the tags from the following list.

* **NOUN**: Noun (e.g., хомяк)
* **ADJF**: Adjective, full form (e.g., хороший)
* **ADJS**: Adjective, short form (e.g., хорош)
* **COMP**: Comparative (e.g., лучше)
* **VERB**: Verb, finite form (e.g., говорил)
* **INFN**: Verb, infinitive (e.g., говорить)
* **PRTF**: Participle, full form (e.g., прочитавший)
* **PRTS**: Participle, short form (e.g.,

Будем использовать JSON-режим, чтобы выходной ответ точно парсился. Можно было бы сделать настоящий и более стабильный structured output, но лень. Также установим температуру пониже, чтобы улучшить точность и предсказуемость.

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": sentence}
    ],
    temperature=0.1,
    response_format={
        'type': 'json_object'
    }
)

In [ ]:
predicted_pos = json.loads(response.choices[0].message.content)

print("Predicted POS tags:")
print(predicted_pos)

if predicted_pos["tokens"] == sentence_pos:
    print("\nSuccess! The predicted tagging exactly matches the ground truth.")
else:
    print("\nFailure. The predicted tagging does not match the ground truth.")

Predicted POS tags:
{'tokens': [['Великолепная', 'ADJF'], ['«', 'PNCT'], ['Школа', 'NOUN'], ['злословия', 'NOUN'], ['»', 'PNCT'], ['вернулась', 'VERB'], ['в', 'PREP'], ['эфир', 'NOUN'], ['после', 'PREP'], ['летних', 'ADJF'], ['каникул', 'NOUN'], ['в', 'PREP'], ['новом', 'ADJF'], ['формате', 'NOUN'], ['.', 'PNCT']]}

Success! The predicted tagging exactly matches the ground truth.


Всё получилось, и даже few-shot не понадобился!

В принципе, промпт достаточно универсальный, чтобы работать на любом предложении, но давайте попробуем что-нибудь сложнее.

> Но давайте зафиксируем: сегодня для практических задач функциональной эмуляции естественного языка и задач, решаемых на компьютере при помощи естественного языка, ЛИНГВИСТИКА НЕ НУЖНА и ЛИНГВИСТЫ ТОЖЕ НЕ НУЖНЫ.

© [Даниил Скоринкин](https://sysblok.ru/blog/gorkij-urok-abbyy-kak-lingvisty-proigrali-poslednjuju-bitvu-za-nlp/)

In [ ]:
sentence2 = """Но давайте зафиксируем: сегодня для практических задач функциональной эмуляции естественного языка и задач, решаемых на компьютере при помощи естественного языка, ЛИНГВИСТИКА НЕ НУЖНА и ЛИНГВИСТЫ ТОЖЕ НЕ НУЖНЫ."""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": sentence2}
    ],
    temperature=0.1,
    response_format={
        'type': 'json_object'
    }
)

In [ ]:
predicted_pos = json.loads(response.choices[0].message.content)

print("Predicted POS tags:")
print(predicted_pos)

Predicted POS tags:
{'tokens': [['Но', 'CONJ'], ['давайте', 'VERB'], ['зафиксируем', 'VERB'], [':', 'PNCT'], ['сегодня', 'ADVB'], ['для', 'PREP'], ['практических', 'ADJF'], ['задач', 'NOUN'], ['функциональной', 'ADJF'], ['эмуляции', 'NOUN'], ['естественного', 'ADJF'], ['языка', 'NOUN'], ['и', 'CONJ'], ['задач', 'NOUN'], [',', 'PNCT'], ['решаемых', 'PRTS'], ['на', 'PREP'], ['компьютере', 'NOUN'], ['при', 'PREP'], ['помощи', 'NOUN'], ['естественного', 'ADJF'], ['языка', 'NOUN'], [',', 'PNCT'], ['ЛИНГВИСТИКА', 'NOUN'], ['НЕ', 'PRCL'], ['НУЖНА', 'VERB'], ['и', 'CONJ'], ['ЛИНГВИСТЫ', 'NOUN'], ['ТОЖЕ', 'ADVB'], ['НЕ', 'PRCL'], ['НУЖНЫ', 'VERB'], ['.', 'PNCT']]}


Здесь уже веселее. Токенизация прошла чисто и в целом разметка тоже не ужасная. Но есть приколы. Во-первых, *решаемых* модель разметила как краткую форму причастия, хотя здесь она всё же полная. Во-вторых, слово *нужный* было размечено как глагол, хотя это скорее краткое прилагательное. Ну и *тоже* — союз, а не наречие. В целом видно, что с чуть более неочевидными кейсами эта модель справляется уже хуже, во всяком случае, без дополнительных примеров. При этом это говорит именно о качестве и знаниях модели, а не о плохой генерализации промпта — в промпте нет ничего, что помогало бы модели в чём-то специфичном, связанным с первым предложением. Оно просто сильно проще.

# Задание 2


Проверьте серию GPT моделей на знание лингвистики. Найдите один хороший глубокий вопрос, с которым не справляется `gpt-4o`. Проверьте, как отвечают на этот вопрос более новые модели: `gpt-4.1`, `o3`, `gpt-5-mini`, `gpt-5`.


Хочу проверить, как модели знают неочевидные лексические функции без дополнительного контекста.

Лексические функции (по Мельчуку) — это слова-операторы, которые выражают какой-то смысл относительно аргумента. Например: MAGN (высокая степень), BON (положительная оценка), INCEP (начало действия), FIN (окончание действия) и т.д.

Есть функция LIQU, смысл который такой: "делать так, чтобы событие перестало иметь место". В отличие от FIN, где действие направлено изнутри, при LIQU нужен какой-то внешний участник, который инициирует прекращение действия. Например, "выпустить из тюрьмы", "возместить ущерб", etc.

Сложный вопрос, который был в одном из заданий на курсе семантики: LIQU от *быть замужем*. Вариант *развестись* не подходит, это действие инициировано изнутри, поэтому это FIN. В итоге правильным ответом могло считаться слово *развести* (в значении 'работник ЗАГСа развёл их'), или *расторгнуть брак* (в том же значении, т.е. когда брак расторгает кто-то извне).

Теперь посмотрим, знают ли модели эту функцию.

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv


load_dotenv()

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
question = "Назови слово или слова, которые соответствуют лексической функции LIQU для 'быть замужем'."

In [ ]:
# GPT-4o
model_name = "gpt-4o"

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": question}]
)

print(f"--- Ответ от {model_name} ---")
print(response.choices[0].message.content)

--- Ответ от gpt-4o ---
Для глагола "быть замужем" лексическая функция LIQU может соответствовать словам "развестись" или "развод".


Что-то знает, но всё же не то, *развестись* — это FIN, а *развод* и вовсе не глагол.

In [ ]:
# GPT-4.1
model_name = "gpt-4.1"

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": question}]
)

print(f"--- Ответ от {model_name} ---")
print(response.choices[0].message.content)

--- Ответ от gpt-4.1 ---
Лексическая функция LIQU (лат. **liquor** — 'жидкость; напиток') обычно указывает на напиток, который традиционно ассоциируется с данным понятием или употребляется в определённой ситуации.

Для значения 'быть замужем' или 'жениться' стандартным примером LIQU является тот напиток, который традиционно употребляют на свадьбах.

**Ответ:**  
Слово, соответствующее лексической функции LIQU для 'быть замужем' в русском языке, — **шампанское**.

**Пояснение:**  
LIQU(Быть замужем) = шампанское,  
так как шампанское традиционно пьют на свадебной церемонии.


> ПРОСТИТЕ НО У МЕНЯ ИСТЕРИКА

In [ ]:
# O3
model_name = "o3"

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": question}]
)

print(f"--- Ответ от {model_name} ---")
print(response.choices[0].message.content)

--- Ответ от o3 ---
Для «быть замужем» значение LIQU реализует усилительную метафору «как в жидкости» – «по уши».  
То есть LIQU(быть замужем) = по уши («Она уже по уши замужем»).


> Жесть

In [ ]:
# GPT-5-mini, с низким ризонингом
model_name = "gpt-5-mini"
reasoning = "low"

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": question}],
    reasoning_effort=reasoning
)

print(f"--- Ответ от {model_name} (Reasoning: {reasoning}) ---")
print(response.choices[0].message.content)

--- Ответ от gpt-5-mini (Reasoning: low) ---
Лексическая функция LIQU обычно даёт глагол(ы), обозначающие вхождение в состояние. Для лексемы «быть замужем» естественные соответствия LIQU:

- выйти замуж (для женщины)  
- жениться / пожениться (для мужчины: жениться на ком-то; пожениться — совершённый акт для пары)

Дополнительно в зависимости от контекста возможны: «состоять в браке» (постоянный статус), «заключить брак» (офиц. акт).


> Ну хотя бы не жидкость... но нет, то, что указала модель — это INCEP

In [ ]:
# GPT-5-mini, с средним ризонингом
model_name = "gpt-5-mini"
reasoning = "medium"

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": question}],
    reasoning_effort=reasoning
)

print(f"--- Ответ от {model_name} (Reasoning: {reasoning}) ---")
print(response.choices[0].message.content)

--- Ответ от gpt-5-mini (Reasoning: medium) ---
Нужно уточнить: что вы понимаете под лексической функцией LIQU? Обычно под LIQU в МТТ понимают функцию «название состояния/фазы» (существительное, обозначающее состояние действия). Если это так, то для предиката «быть замужем» подходят, например:

- замужество  
- брак  
- супружество  
- семейное положение (замужем)  

Если вы имели в виду другую интерпретацию LIQU — напишите, пожалуйста, и я подберу варианты точнее.


> Не знаю, что такое МТТ. Сначала подумалось, что он знает про "смысл-текст", но, видимо, нет — потому что даже не понял, что там должен был быть глагол. Но мы хотя бы поняли, что модель точно не знает функцию LIQU.

In [ ]:
# GPT-5-mini, с высоким ризонингом
model_name = "gpt-5-mini"
reasoning = "high"

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": question}],
    reasoning_effort=reasoning
)

print(f"--- Ответ от {model_name} (Reasoning: {reasoning}) ---")
print(response.choices[0].message.content)

--- Ответ от gpt-5-mini (Reasoning: high) ---
Предположу, что под LIQU вы понимаете лексическую функцию, означающую действие, вводящее в данное состояние (инцептив). В этом случае для «быть замужем» соответствуют, например:

- выйти замуж, вступить в брак, пожениться (для женщины обычно «выйти замуж», для мужчины — «жениться/пожениться»).

(Противоположное действие — прекращение состояния: «развестись».)


> ДА ПОЧЕМУ ИНЦЕПТИВ-ТО

In [ ]:
# GPT-5, с низким ризонингом
model_name = "gpt-5"
reasoning = "low"

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": question}],
    reasoning_effort=reasoning
)

print(f"--- Ответ от {model_name} (Reasoning: {reasoning}) ---")
print(response.choices[0].message.content)

--- Ответ от gpt-5 (Reasoning: low) ---
Развестись; расторгнуть брак.


> Ну почти, *расторгнуть брак*, кстати, вполне валидно в некоторых контекстах!

In [ ]:
# gpt-5, с средним ризонингом
model_name = "gpt-5"
reasoning = "medium"

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": question}],
    reasoning_effort=reasoning
)

print(f"--- Ответ от {model_name} (Reasoning: {reasoning}) ---")
print(response.choices[0].message.content)

--- Ответ от gpt-5 (Reasoning: medium) ---
Развестись; расторгнуть (аннулировать) брак.


> А вот это здорово! *Развестись* всё ещё мимо, но с уточнением про "аннулировать" *расторгнуть брак* подходит!

In [ ]:
# gpt-5, с высоким ризонингом
model_name = "gpt-5"
reasoning = "high"

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": question}],
    reasoning_effort=reasoning
)

print(f"--- Ответ от {model_name} (Reasoning: {reasoning}) ---")
print(response.choices[0].message.content)

--- Ответ от gpt-5 (Reasoning: high) ---
- развестись (разводиться)
- овдоветь


> Средний ризонинг справился лучше, этот зачем-то просто привёл видовую пару.

Можно сказать, что модели в основном не знают, о чём идёт речь, когда я упоминаю лексическую функцию LIQU. Хотя бы приблизительное понимание демонстрирует модель 4O и GPT-5, GPT-5-mini уходит в противоположную сторону, но хотя бы остаётся во фреймворке "смысл-текст", а вот 4.1 и O3 очень развеселили, начав по-настоящему фантазировать.

Похоже, что всё упирается в знания модели: чем она больше по кол-ву параметров, тем больше вероятность, что ей известно хоть что-то про лексические функции. Про 4.1 известно, что она в целом глупее 4O (но быстрее), а вот O3 с ризонингом прям разочаровала. Ещё видно, что уровень ризонинга, который можно настроить для 5 и 5-Mini, влияет на ответ, но слишком много ризонинга тоже плохо, как и слишком мало: то есть самые близкие к правильным ответы получались на среднем уровне.

Да, конечно, тут надо учитывать, что в таком режиме модель не могла использовать веб-поиск и в принципе ориентировалась исключительно на свои знания, я нарочно не дал вообще никакого контекста. В реальной жизни, конечно, я бы так никогда не делал. Собственно, когда я учился на бакалавриате ФиКЛа и активно делал домашки и КР нейронками, подсовывание оригинальных презентаций, конспектов и других материалов давало отличный результат в таких лингвистических вопросах.

# Задание 3

Проверьте тот же самый вопрос на любой из моделей Qwen 3 (например той что мы использовали на семинаре)

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
question = "Назови слово или слова, которые соответствуют лексической функции LIQU для 'быть замужем'."

In [3]:
# Версия без ризонинга
model_name_instruct = "Qwen/Qwen3-4B-Instruct-2507"

tokenizer_instruct = AutoTokenizer.from_pretrained(model_name_instruct)
model_instruct = AutoModelForCausalLM.from_pretrained(
    model_name_instruct,
    torch_dtype="auto",
    device_map="auto"
)

messages_instruct = [
    {"role": "user", "content": question}
]
text_instruct = tokenizer_instruct.apply_chat_template(
    messages_instruct,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs_instruct = tokenizer_instruct(
    [text_instruct], return_tensors="pt").to(
        model_instruct.device)

generated_ids_instruct = model_instruct.generate(
    **model_inputs_instruct,
    max_new_tokens=2048
)
output_ids_instruct = generated_ids_instruct[0][len(
    model_inputs_instruct.input_ids[0]):].tolist()

content_instruct = tokenizer_instruct.decode(
    output_ids_instruct, skip_special_tokens=True)

print(f"--- Ответ от {model_name_instruct} ---")
print(content_instruct)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

--- Ответ от Qwen/Qwen3-4B-Instruct-2507 ---
Слово или слова, которые соответствуют лексической функции **LIQU** (от англ. *Linguistic Function* — лексическая функция) для выражения **"быть замужем"**, — это **"замужем"**.

### Объяснение:
- **LIQU** — это лексическая функция, которая обозначает **семантическое значение**, связанное с **статусом или состоянием** (например, быть замужем, быть студентом, быть врачом и т.д.).
- Слово **"замужем"** выражает **социальный статус** или **семейное состояние**.
- Оно используется как **прилагательное**, описывает состояние человека в отношении брака.

### Пример:
- Он **замужем** — это утверждение о текущем семейном положении.

Таким образом, слово, соответствующее лексической функции **LIQU** для "быть замужем", — это:

> **замужем**

✅ Это слово отражает **статус** (функция LIQU — статусное значение), характерное для лексических функций в лингвистике.

---

🔹 *Примечание:* В лингвистике (особенно в теории лексической функции) LIQU может означ

Собственно, ожидаемо. Модель не просто не знает, что такое LIQU - она в принципе не поняла, что такое лексическая функция.

In [3]:
# Версия с ризонингом
model_name_thinking = "Qwen/Qwen3-4B-Thinking-2507"

tokenizer_thinking = AutoTokenizer.from_pretrained(model_name_thinking)
model_thinking = AutoModelForCausalLM.from_pretrained(
    model_name_thinking,
    torch_dtype="auto",
    device_map="auto"
)

messages_thinking = [
    {"role": "user", "content": question}
]
text_thinking = tokenizer_thinking.apply_chat_template(
    messages_thinking,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs_thinking = tokenizer_thinking(
    [text_thinking], return_tensors="pt").to(
        model_thinking.device)

generated_ids_thinking = model_thinking.generate(
    **model_inputs_thinking,
    max_new_tokens=16384
)
output_ids_thinking = generated_ids_thinking[0][len(
    model_inputs_thinking.input_ids[0]):].tolist()

try:
    index = len(output_ids_thinking) - output_ids_thinking[::-1].index(151668)
except ValueError:
    index = 0

reasoning_content_thinking = tokenizer_thinking.decode(
    output_ids_thinking[:index], skip_special_tokens=True).strip("\n")
content_thinking = tokenizer_thinking.decode(
    output_ids_thinking[index:], skip_special_tokens=True).strip("\n")

print(f"--- Рассуждения от {model_name_thinking} ---")
print(reasoning_content_thinking)
print(f"--- Ответ от {model_name_thinking} ---")
print(content_thinking)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

--- Рассуждения от Qwen/Qwen3-4B-Thinking-2507 ---
Хорошо, мне нужно определить, какие слова или словосочетания соответствуют лексической функции LIQU для глагола "быть замужем". Сначала я должен вспомнить, что такое LIQU. 

LIQU — это лексическая функция в лингвистике, которая обозначает "лику" (от лат. liquidus), но, возможно, здесь имеется в виду что-то другое. Нет, подожди, в контексте русского языка или общего лингвистического анализа LIQU может относиться к определенной категории. 

Подожду, возможно, это опечатка или неправильный термин. В русском языке есть лексические функции, например, глаголы, существительные, прилагательные и т.д. Но LIQU не является стандартной лексической функцией в русском языке. Может быть, это термин из другой лингвистики, например, в английском языке есть лексические функции, но LIQU не входит в стандартные. 

Допустим, что это опечатка. Например, возможно, имелось в виду "лику" как часть другого термина. Или, может быть, это связано с функцией "лику"

Рассуждения доставляют! Хотя она, кажется, несколько зациклилась. Но все рассуждения сводятся к тому, чтобы вообще понять, что такое лексическая функция в целом и LIQU в частности, очевидно, таких знаний у этих маленьких моделей вообще нет.

Опять же: я не играл с гиперпараметрами, у нас не было поиска (а ризонинг-модель, судя по рассуждениям, очень хотела бы полазить в интернете), но в целом ясно, что для подобных вопросов нужны модели сильно большего размера.

# Задание 4

Выберите 3 языка, которые очень мало похожи на английский язык и которые мало похожи между собой. Используя токенизатор Qwen3 рассчитайте среднее отношение количества токенов к количеству символов для каждого языка. Для этого найдите какой-нибудь текст длинной более 5000 символов (или просто несколько текстов) и токенизируйте его. Поделите общее количество токенов на общее количество символов.

Возьмём один и тот же текст на трёх языках. Пусть это будет «Всеобщая декларация прав человека» ООН на русском, греческом и китайском.

In [ ]:
from transformers import AutoTokenizer

In [ ]:
pretrained_model = "Qwen/Qwen3-4B-Thinking-2507"

tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model,
    use_fast=False,
    trust_remote_code=True
)

In [ ]:
def analyze_language_tokenization(file_path, tokenizer):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    num_chars = len(text)
    num_tokens = len(tokenizer.encode(text, add_special_tokens=False))

    ratio = num_tokens / num_chars if num_chars > 0 else 0

    print(f"--- Analysis for: {file_path} ---")
    print(f"Total characters: {num_chars}")
    print(f"Total tokens: {num_tokens}")
    print(f"Token-to-character ratio: {ratio:.4f}")
    if ratio > 0:
        print(
            f"(This means there is approximately 1 token for every {1/ratio:.2f} characters)")

In [ ]:
# Русский
analyze_language_tokenization('hw5_ru.txt', tokenizer)

--- Analysis for: hw5_ru.txt ---
Total characters: 11774
Total tokens: 3524
Token-to-character ratio: 0.2993
(This means there is approximately 1 token for every 3.34 characters)


In [ ]:
# Греческий
analyze_language_tokenization('hw5_el.txt', tokenizer)

--- Analysis for: hw5_el.txt ---
Total characters: 12461
Total tokens: 10617
Token-to-character ratio: 0.8520
(This means there is approximately 1 token for every 1.17 characters)


In [ ]:
# Китайский
analyze_language_tokenization('hw5_zh.txt', tokenizer)

--- Analysis for: hw5_zh.txt ---
Total characters: 3013
Total tokens: 1926
Token-to-character ratio: 0.6392
(This means there is approximately 1 token for every 1.56 characters)


Можно видеть, что греческий язык токенизируется совсем неэффективно, несмотря на то, что это флективный язык, как и русский. Очень часто один токен соответствует одному символу. Эффективность токенизации для русского относительно неплохая, почти 3.5 символов на токен в среднем, но, кажется, это всё ещё далеко от такой, при которой токен стремится к слову (хотя для русского оно и не надо, кажется). Наконец, если смотреть чисто на отношение, эффективность для китайского хуже, чем для русского, однако за счёт того, что китайский текст не содержит пробелов и в целом содержит раза в четыре меньше символов, чем русский или греческий, для него вполне нормально, когда токенизация стремиться к посимвольной, выучивая лишь некоторые мультисимвольные паттерны.